<a href="https://colab.research.google.com/github/914book/FinalProjectGroupD/blob/sam_wk4/Group4_ML_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
from sqlalchemy import create_engine
from getpass import getpass
import numpy as np 
import sklearn as skl
import math
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import r2_score
from tensorflow.keras import layers
from sklearn.ensemble import RandomForestClassifier

secret = getpass('Enter the secret value: ')

Enter the secret value: ··········


In [2]:
#pip install psycopg2

In [3]:
import psycopg2
connection = psycopg2.connect(
    host = 'housingdatabase.chkn2wwlsbys.us-east-2.rds.amazonaws.com',
    port = 5432,
    user = 'postgres',
    password = secret,
    database='NYChouses'
    )
cursor=connection.cursor()


/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
import pandas as pd

pd.set_option('display.max_colwidth',1000)

In [5]:
sql = """
SELECT *
FROM fulloutputhousing
"""
fulloutpuhousing = pd.read_sql(sql, con=connection)

In [6]:
#table for M/L model based of test data share with Toral
house2data = fulloutpuhousing[['house_price', 'address', 'zipcode', 'date', 'beds', 'bath','sq_ft', 'boro', 'employed', 'unemployed', 'income_over_100k', 'income_under_100k']]
house2data.columns = ['price','address','zip','date','beds','bath','sq_ft','boro','Employed','Unemployed','income above 100k','income below 100k']
house2data


,price,address,zip,date,beds,bath,sq_ft,boro,Employed,Unemployed,income above 100k,income below 100k
0,244000,"4205 48th Ave APT 3G, Flushing, NY 11377",11377,2020-10-16,Studio,1.0,53400,Queens County,1099996,39401,166650,1015040
1,680000,"14404 76th Rd, Flushing, NY 11367",11367,2020-10-16,3,3.0,1980,Queens County,1099996,39401,166650,1015040
2,900000,"35 88th St, Brooklyn, NY 11209",11209,2020-10-16,3,1.0,1520,Kings County,1149461,48699,188640,1067440
3,625000,"11128 130th St, Jamaica, NY 11420",11420,2020-10-16,3,2.0,1440,Queens County,1099996,39401,166650,1015040
4,1720000,"20102 32nd Ave, Flushing, NY 11361",11361,2020-10-16,2,1.0,1406,Queens County,1099996,39401,166650,1015040
...,...,...,...,...,...,...,...,...,...,...,...,...
72657,317554,"2835 Webb Ave, Bronx, NY 10468",10468,2020-09-11,2,2.0,83000,Bronx County,565642,32035,55180,173430
72658,157000,"2640 Marion Ave APT 6H, Bronx, NY 10458",10458,2020-08-18,2,1.0,91075,Bronx County,565642,32035,55180,173430
72659,11500,"1711 Davidson Ave, Bronx, NY 10453",10453,2020-08-07,0,0.0,98700,Bronx County,565642,32035,55180,173430
72660,180887,"871 Elton Ave APT 2, Bronx, NY 10451",10451,2020-08-24,0,0.0,99240,Bronx County,565642,32035,55180,173430


In [7]:
house2data.groupby('boro').mean()

,price,zip,bath,sq_ft,Employed,Unemployed,income above 100k,income below 100k
boro,,,,,,,,
Bronx County,5.021488e+05,10464.877898,1.732945,3375.714065,565642.0,32035.0,55180.0,173430.0
Kings County,1.415096e+06,11228.788652,1.635106,3836.717730,1149461.0,48699.0,188640.0,1067440.0
New York County,2.200729e+06,10019.301713,1.467185,6219.493390,879665.0,31973.0,300250.0,594060.0
Queens County,7.026108e+05,11389.757455,1.862737,4115.249912,1099996.0,39401.0,166650.0,1015040.0
Richmond County,5.824694e+05,10308.421703,2.323308,1830.938187,213734.0,8585.0,55180.0,173430.0


In [8]:
df = house2data
df.head()

,price,address,zip,date,beds,bath,sq_ft,boro,Employed,Unemployed,income above 100k,income below 100k
0,244000,"4205 48th Ave APT 3G, Flushing, NY 11377",11377,2020-10-16,Studio,1.0,53400,Queens County,1099996,39401,166650,1015040
1,680000,"14404 76th Rd, Flushing, NY 11367",11367,2020-10-16,3,3.0,1980,Queens County,1099996,39401,166650,1015040
2,900000,"35 88th St, Brooklyn, NY 11209",11209,2020-10-16,3,1.0,1520,Kings County,1149461,48699,188640,1067440
3,625000,"11128 130th St, Jamaica, NY 11420",11420,2020-10-16,3,2.0,1440,Queens County,1099996,39401,166650,1015040
4,1720000,"20102 32nd Ave, Flushing, NY 11361",11361,2020-10-16,2,1.0,1406,Queens County,1099996,39401,166650,1015040


In [9]:
# Print out the boro value counts
boro_counts = df.boro.value_counts()
boro_counts

Queens County                                         45608
New York County                                       21484
Kings County                                           2820
Richmond County                                        1456
Bronx County                                           1294
Name: boro, dtype: int64

In [10]:
new_df = df.drop(['address', 'date'], axis=1)
new_df.head()


,price,zip,beds,bath,sq_ft,boro,Employed,Unemployed,income above 100k,income below 100k
0,244000,11377,Studio,1.0,53400,Queens County,1099996,39401,166650,1015040
1,680000,11367,3,3.0,1980,Queens County,1099996,39401,166650,1015040
2,900000,11209,3,1.0,1520,Kings County,1149461,48699,188640,1067440
3,625000,11420,3,2.0,1440,Queens County,1099996,39401,166650,1015040
4,1720000,11361,2,1.0,1406,Queens County,1099996,39401,166650,1015040


In [11]:
new_df['total_emp'] = new_df['Employed'] + new_df['Unemployed']
new_df.head()

,price,zip,beds,bath,sq_ft,boro,Employed,Unemployed,income above 100k,income below 100k,total_emp
0,244000,11377,Studio,1.0,53400,Queens County,1099996,39401,166650,1015040,1139397
1,680000,11367,3,3.0,1980,Queens County,1099996,39401,166650,1015040,1139397
2,900000,11209,3,1.0,1520,Kings County,1149461,48699,188640,1067440,1198160
3,625000,11420,3,2.0,1440,Queens County,1099996,39401,166650,1015040,1139397
4,1720000,11361,2,1.0,1406,Queens County,1099996,39401,166650,1015040,1139397


In [12]:
new_df['Unemployment_rate'] = new_df['Unemployed']/new_df['total_emp']
new_df.head()

,price,zip,beds,bath,sq_ft,boro,Employed,Unemployed,income above 100k,income below 100k,total_emp,Unemployment_rate
0,244000,11377,Studio,1.0,53400,Queens County,1099996,39401,166650,1015040,1139397,0.034581
1,680000,11367,3,3.0,1980,Queens County,1099996,39401,166650,1015040,1139397,0.034581
2,900000,11209,3,1.0,1520,Kings County,1149461,48699,188640,1067440,1198160,0.040645
3,625000,11420,3,2.0,1440,Queens County,1099996,39401,166650,1015040,1139397,0.034581
4,1720000,11361,2,1.0,1406,Queens County,1099996,39401,166650,1015040,1139397,0.034581


In [13]:
newrate_df = new_df.drop(['Employed', 'Unemployed', 'total_emp'], axis=1)
newrate_df.head()

,price,zip,beds,bath,sq_ft,boro,income above 100k,income below 100k,Unemployment_rate
0,244000,11377,Studio,1.0,53400,Queens County,166650,1015040,0.034581
1,680000,11367,3,3.0,1980,Queens County,166650,1015040,0.034581
2,900000,11209,3,1.0,1520,Kings County,188640,1067440,0.040645
3,625000,11420,3,2.0,1440,Queens County,166650,1015040,0.034581
4,1720000,11361,2,1.0,1406,Queens County,166650,1015040,0.034581


In [14]:
newrate_df['income2'] = newrate_df['income above 100k'] + newrate_df['income below 100k']
newrate_df.head()

,price,zip,beds,bath,sq_ft,boro,income above 100k,income below 100k,Unemployment_rate,income2
0,244000,11377,Studio,1.0,53400,Queens County,166650,1015040,0.034581,1181690
1,680000,11367,3,3.0,1980,Queens County,166650,1015040,0.034581,1181690
2,900000,11209,3,1.0,1520,Kings County,188640,1067440,0.040645,1256080
3,625000,11420,3,2.0,1440,Queens County,166650,1015040,0.034581,1181690
4,1720000,11361,2,1.0,1406,Queens County,166650,1015040,0.034581,1181690


In [15]:
newrate_df['income_rate'] = newrate_df['income above 100k']/newrate_df['income2']
newrate_df.head()

,price,zip,beds,bath,sq_ft,boro,income above 100k,income below 100k,Unemployment_rate,income2,income_rate
0,244000,11377,Studio,1.0,53400,Queens County,166650,1015040,0.034581,1181690,0.141027
1,680000,11367,3,3.0,1980,Queens County,166650,1015040,0.034581,1181690,0.141027
2,900000,11209,3,1.0,1520,Kings County,188640,1067440,0.040645,1256080,0.150182
3,625000,11420,3,2.0,1440,Queens County,166650,1015040,0.034581,1181690,0.141027
4,1720000,11361,2,1.0,1406,Queens County,166650,1015040,0.034581,1181690,0.141027


In [16]:
newrates_df = newrate_df.drop(['income above 100k', 'income below 100k', 'income2'], axis=1)
newrates_df.head()

,price,zip,beds,bath,sq_ft,boro,Unemployment_rate,income_rate
0,244000,11377,Studio,1.0,53400,Queens County,0.034581,0.141027
1,680000,11367,3,3.0,1980,Queens County,0.034581,0.141027
2,900000,11209,3,1.0,1520,Kings County,0.040645,0.150182
3,625000,11420,3,2.0,1440,Queens County,0.034581,0.141027
4,1720000,11361,2,1.0,1406,Queens County,0.034581,0.141027


In [17]:
newrates_df.loc[newrate_df.beds == "Studio", "beds"] = "0"
newrates_df.head()

,price,zip,beds,bath,sq_ft,boro,Unemployment_rate,income_rate
0,244000,11377,0,1.0,53400,Queens County,0.034581,0.141027
1,680000,11367,3,3.0,1980,Queens County,0.034581,0.141027
2,900000,11209,3,1.0,1520,Kings County,0.040645,0.150182
3,625000,11420,3,2.0,1440,Queens County,0.034581,0.141027
4,1720000,11361,2,1.0,1406,Queens County,0.034581,0.141027


In [18]:
# Create the OneHotEncoder instance
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False,drop ="first")

In [19]:
# Fit the encoder and produce encoded DataFrame
boro_df = pd.DataFrame(enc.fit_transform(newrates_df.boro.values.reshape(-1,1)))

In [20]:
# Rename encoded columns
boro_df.columns = enc.get_feature_names(['boro'])
boro_df.head()

,boro_Kings County,boro_New York County,boro_Queens County,boro_Richmond County
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0


In [21]:
# Merge the two DataFrames together and drop the boro column
new2_df=newrates_df.merge(boro_df,left_index=True,right_index=True).drop("boro",1)
new2_df.head()

,price,zip,beds,bath,sq_ft,Unemployment_rate,income_rate,boro_Kings County,boro_New York County,boro_Queens County,boro_Richmond County
0,244000,11377,0,1.0,53400,0.034581,0.141027,0.0,0.0,1.0,0.0
1,680000,11367,3,3.0,1980,0.034581,0.141027,0.0,0.0,1.0,0.0
2,900000,11209,3,1.0,1520,0.040645,0.150182,1.0,0.0,0.0,0.0
3,625000,11420,3,2.0,1440,0.034581,0.141027,0.0,0.0,1.0,0.0
4,1720000,11361,2,1.0,1406,0.034581,0.141027,0.0,0.0,1.0,0.0


In [22]:

#Find null Values

for column in new2_df.columns:
    print(f"Column {column} has {new2_df[column].isnull().sum()} null values")

Column price has 0 null values
Column zip has 0 null values
Column beds has 0 null values
Column bath has 218 null values
Column sq_ft has 0 null values
Column Unemployment_rate has 0 null values
Column income_rate has 0 null values
Column boro_Kings County                                       has 0 null values
Column boro_New York County                                    has 0 null values
Column boro_Queens County                                      has 0 null values
Column boro_Richmond County                                    has 0 null values


In [23]:
# Drop null rows
new2_df = new2_df.dropna()


In [24]:
#Find null Values

for column in new2_df.columns:
    print(f"Column {column} has {new2_df[column].isnull().sum()} null values")

Column price has 0 null values
Column zip has 0 null values
Column beds has 0 null values
Column bath has 0 null values
Column sq_ft has 0 null values
Column Unemployment_rate has 0 null values
Column income_rate has 0 null values
Column boro_Kings County                                       has 0 null values
Column boro_New York County                                    has 0 null values
Column boro_Queens County                                      has 0 null values
Column boro_Richmond County                                    has 0 null values


In [25]:
# List dataframe data types
new2_df.dtypes

price                                                        int64
zip                                                          int64
beds                                                        object
bath                                                       float64
sq_ft                                                        int64
Unemployment_rate                                          float64
income_rate                                                float64
boro_Kings County                                          float64
boro_New York County                                       float64
boro_Queens County                                         float64
boro_Richmond County                                       float64
dtype: object

In [26]:
new2_df['zip']= new2_df['zip'].astype('str')
new2_df.head()

,price,zip,beds,bath,sq_ft,Unemployment_rate,income_rate,boro_Kings County,boro_New York County,boro_Queens County,boro_Richmond County
0,244000,11377,0,1.0,53400,0.034581,0.141027,0.0,0.0,1.0,0.0
1,680000,11367,3,3.0,1980,0.034581,0.141027,0.0,0.0,1.0,0.0
2,900000,11209,3,1.0,1520,0.040645,0.150182,1.0,0.0,0.0,0.0
3,625000,11420,3,2.0,1440,0.034581,0.141027,0.0,0.0,1.0,0.0
4,1720000,11361,2,1.0,1406,0.034581,0.141027,0.0,0.0,1.0,0.0


In [27]:
# Create the OneHotEncoder instance
from sklearn.preprocessing import OneHotEncoder
encz = OneHotEncoder(sparse=False,drop ="first")

In [28]:
# Fit the encoder and produce encoded DataFrame
zipone_df = pd.DataFrame(encz.fit_transform(new2_df.zip.values.reshape(-1,1)))

In [29]:
# Rename encoded columns
zipone_df.columns = encz.get_feature_names(['zip'])
zipone_df.head()

,zip_10002,zip_10003,zip_10004,zip_10005,zip_10006,zip_10007,zip_10009,zip_10010,zip_10011,zip_10012,zip_10013,zip_10014,zip_10016,zip_10017,zip_10018,zip_10019,zip_10021,zip_10022,zip_10023,zip_10024,zip_10025,zip_10026,zip_10027,zip_10028,zip_10029,zip_10030,zip_10031,zip_10032,zip_10033,zip_10034,zip_10035,zip_10036,zip_10037,zip_10038,zip_10039,zip_10040,zip_10044,zip_10065,zip_10075,zip_10128,...,zip_11362,zip_11363,zip_11364,zip_11365,zip_11366,zip_11367,zip_11368,zip_11369,zip_11370,zip_11372,zip_11373,zip_11374,zip_11375,zip_11377,zip_11378,zip_11379,zip_11385,zip_11411,zip_11412,zip_11413,zip_11414,zip_11415,zip_11416,zip_11417,zip_11418,zip_11419,zip_11420,zip_11421,zip_11422,zip_11423,zip_11426,zip_11427,zip_11428,zip_11429,zip_11430,zip_11432,zip_11433,zip_11434,zip_11435,zip_11436
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
# Merge the two DataFrames together and drop the boro column
new3_df=new2_df.merge(zipone_df,left_index=True,right_index=True).drop("zip",1)
new3_df.head()

,price,beds,bath,sq_ft,Unemployment_rate,income_rate,boro_Kings County,boro_New York County,boro_Queens County,boro_Richmond County,zip_10002,zip_10003,zip_10004,zip_10005,zip_10006,zip_10007,zip_10009,zip_10010,zip_10011,zip_10012,zip_10013,zip_10014,zip_10016,zip_10017,zip_10018,zip_10019,zip_10021,zip_10022,zip_10023,zip_10024,zip_10025,zip_10026,zip_10027,zip_10028,zip_10029,zip_10030,zip_10031,zip_10032,zip_10033,zip_10034,...,zip_11362,zip_11363,zip_11364,zip_11365,zip_11366,zip_11367,zip_11368,zip_11369,zip_11370,zip_11372,zip_11373,zip_11374,zip_11375,zip_11377,zip_11378,zip_11379,zip_11385,zip_11411,zip_11412,zip_11413,zip_11414,zip_11415,zip_11416,zip_11417,zip_11418,zip_11419,zip_11420,zip_11421,zip_11422,zip_11423,zip_11426,zip_11427,zip_11428,zip_11429,zip_11430,zip_11432,zip_11433,zip_11434,zip_11435,zip_11436
0,244000,0,1.0,53400,0.034581,0.141027,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,680000,3,3.0,1980,0.034581,0.141027,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,900000,3,1.0,1520,0.040645,0.150182,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,625000,3,2.0,1440,0.034581,0.141027,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1720000,2,1.0,1406,0.034581,0.141027,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
new3_df.dtypes

price                  int64
beds                  object
bath                 float64
sq_ft                  int64
Unemployment_rate    float64
                      ...   
zip_11432            float64
zip_11433            float64
zip_11434            float64
zip_11435            float64
zip_11436            float64
Length: 168, dtype: object

In [32]:
new3_df['beds']= new3_df['beds'].astype('int')
new3_df.head()

,price,beds,bath,sq_ft,Unemployment_rate,income_rate,boro_Kings County,boro_New York County,boro_Queens County,boro_Richmond County,zip_10002,zip_10003,zip_10004,zip_10005,zip_10006,zip_10007,zip_10009,zip_10010,zip_10011,zip_10012,zip_10013,zip_10014,zip_10016,zip_10017,zip_10018,zip_10019,zip_10021,zip_10022,zip_10023,zip_10024,zip_10025,zip_10026,zip_10027,zip_10028,zip_10029,zip_10030,zip_10031,zip_10032,zip_10033,zip_10034,...,zip_11362,zip_11363,zip_11364,zip_11365,zip_11366,zip_11367,zip_11368,zip_11369,zip_11370,zip_11372,zip_11373,zip_11374,zip_11375,zip_11377,zip_11378,zip_11379,zip_11385,zip_11411,zip_11412,zip_11413,zip_11414,zip_11415,zip_11416,zip_11417,zip_11418,zip_11419,zip_11420,zip_11421,zip_11422,zip_11423,zip_11426,zip_11427,zip_11428,zip_11429,zip_11430,zip_11432,zip_11433,zip_11434,zip_11435,zip_11436
0,244000,0,1.0,53400,0.034581,0.141027,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,680000,3,3.0,1980,0.034581,0.141027,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,900000,3,1.0,1520,0.040645,0.150182,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,625000,3,2.0,1440,0.034581,0.141027,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1720000,2,1.0,1406,0.034581,0.141027,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
new3_df.dtypes

price                  int64
beds                   int64
bath                 float64
sq_ft                  int64
Unemployment_rate    float64
                      ...   
zip_11432            float64
zip_11433            float64
zip_11434            float64
zip_11435            float64
zip_11436            float64
Length: 168, dtype: object

In [34]:
# Print out the bath value counts
bath_counts = new3_df.bath.value_counts()
bath_counts

1.00     23310
2.00     18484
0.00     11973
3.00     10954
4.00      3267
5.00      1550
1.50       892
2.50       569
6.00       488
7.00       163
3.50       159
11.00       90
12.00       80
4.50        62
8.00        55
9.00        31
10.00       28
13.00       16
1.75        14
5.50        10
21.00        8
22.00        6
23.00        6
14.00        6
8.50         4
6.50         4
20.00        4
2.75         2
0.50         2
1.10         2
25.00        2
24.00        2
16.00        2
1.25         2
26.00        2
Name: bath, dtype: int64

In [35]:
# Print out the bath value counts
price_counts = new3_df.price.value_counts()

price_counts

10        916
750000    652
800000    636
650000    615
550000    592
         ... 
317554      1
11500       1
943400      1
834502      1
725237      1
Name: price, Length: 3744, dtype: int64

In [36]:
delete_row = new3_df[new3_df["price"]<11].index
new3_df = new3_df.drop(delete_row)
new3_df.head()

,price,beds,bath,sq_ft,Unemployment_rate,income_rate,boro_Kings County,boro_New York County,boro_Queens County,boro_Richmond County,zip_10002,zip_10003,zip_10004,zip_10005,zip_10006,zip_10007,zip_10009,zip_10010,zip_10011,zip_10012,zip_10013,zip_10014,zip_10016,zip_10017,zip_10018,zip_10019,zip_10021,zip_10022,zip_10023,zip_10024,zip_10025,zip_10026,zip_10027,zip_10028,zip_10029,zip_10030,zip_10031,zip_10032,zip_10033,zip_10034,...,zip_11362,zip_11363,zip_11364,zip_11365,zip_11366,zip_11367,zip_11368,zip_11369,zip_11370,zip_11372,zip_11373,zip_11374,zip_11375,zip_11377,zip_11378,zip_11379,zip_11385,zip_11411,zip_11412,zip_11413,zip_11414,zip_11415,zip_11416,zip_11417,zip_11418,zip_11419,zip_11420,zip_11421,zip_11422,zip_11423,zip_11426,zip_11427,zip_11428,zip_11429,zip_11430,zip_11432,zip_11433,zip_11434,zip_11435,zip_11436
0,244000,0,1.0,53400,0.034581,0.141027,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,680000,3,3.0,1980,0.034581,0.141027,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,900000,3,1.0,1520,0.040645,0.150182,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,625000,3,2.0,1440,0.034581,0.141027,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1720000,2,1.0,1406,0.034581,0.141027,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
scaled_features = new3_df.copy()

In [38]:
col_names = ['beds', 'bath','sq_ft', 'Unemployment_rate', 'income_rate']
features = scaled_features[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)

In [39]:
scaled_features[col_names] = features
scaled_features

,price,beds,bath,sq_ft,Unemployment_rate,income_rate,boro_Kings County,boro_New York County,boro_Queens County,boro_Richmond County,zip_10002,zip_10003,zip_10004,zip_10005,zip_10006,zip_10007,zip_10009,zip_10010,zip_10011,zip_10012,zip_10013,zip_10014,zip_10016,zip_10017,zip_10018,zip_10019,zip_10021,zip_10022,zip_10023,zip_10024,zip_10025,zip_10026,zip_10027,zip_10028,zip_10029,zip_10030,zip_10031,zip_10032,zip_10033,zip_10034,...,zip_11362,zip_11363,zip_11364,zip_11365,zip_11366,zip_11367,zip_11368,zip_11369,zip_11370,zip_11372,zip_11373,zip_11374,zip_11375,zip_11377,zip_11378,zip_11379,zip_11385,zip_11411,zip_11412,zip_11413,zip_11414,zip_11415,zip_11416,zip_11417,zip_11418,zip_11419,zip_11420,zip_11421,zip_11422,zip_11423,zip_11426,zip_11427,zip_11428,zip_11429,zip_11430,zip_11432,zip_11433,zip_11434,zip_11435,zip_11436
0,244000,-1.135135,-0.518575,2.739442,-0.286425,-0.698632,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,680000,0.361618,0.854592,-0.153198,-0.286425,-0.698632,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,900000,0.361618,-0.518575,-0.179076,2.044534,-0.595246,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,625000,0.361618,0.168009,-0.183576,-0.286425,-0.698632,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1720000,-0.137300,-0.518575,-0.185489,-0.286425,-0.698632,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72439,640000,-1.135135,-1.205159,-0.123608,7.023896,0.434588,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72440,420000,1.858370,0.854592,-0.123496,7.023896,0.434588,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72441,485000,0.860535,0.168009,-0.123045,1.264579,0.434588,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [40]:
# Split our preprocessed data into our features and target arrays
y = scaled_features["price"].values
X = scaled_features.drop(["price"],1).values

In [41]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

Train and Evaluate Random Forest Classifier

In [42]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error 

In [43]:
# Create a random forest classifier.
rf_model = RandomForestRegressor(n_estimators=128,random_state=78)

In [44]:
# Fitting the model
rf_model = rf_model.fit(X_train, y_train)

In [45]:
# Evaluate the model
y_pred = rf_model.predict(X_test)
value = mean_squared_error(y_test,y_pred)
# calculate square
square = math.sqrt(value)
print(f" Housing price predictive root mean squared error: {square:.3f}")

 Housing price predictive root mean squared error: 1754290.770


In [46]:
value = mean_squared_error(y_test,y_pred)
# calculate square
square = math.sqrt(value)
square

1754290.7700338224

In [ ]:
# get importance
importance = rf_model.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

In [59]:
predictions2 = rf_model.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
0,2.121015e+06,1900000
1,5.629425e+05,516000
2,5.596251e+05,585000
3,7.217891e+05,650000
4,5.153125e+06,5800000
...,...,...
17784,1.250003e+06,935000
17785,1.445686e+06,975000
17786,6.164656e+05,995000
17787,2.352266e+06,1740000


Linear Regression

In [60]:
from sklearn.linear_model import LinearRegression
classifier = LinearRegression()
classifier

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [50]:
classifier.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [51]:
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
0,2926344.0,1900000
1,277968.0,516000
2,740664.0,585000
3,819552.0,650000
4,3324880.0,5800000
...,...,...
17784,1776632.0,935000
17785,2569704.0,975000
17786,823000.0,995000
17787,2629200.0,1740000


In [145]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 443291.23691242904
Mean Squared Error: 3077536105825.8613
Root Mean Squared Error: 1754290.7700338224


In [ ]:
#import pickle

In [ ]:
#pickle.dump(rf_model, open('model.pkl','wb'))
#rf_model = pickle.load(open('model.pkl','rb'))

nn


In [238]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  300
hidden_nodes_layer2 = 128
hidden_nodes_layer3 = 48
hidden_nodes_layer4 = 24


In [239]:
nn_model = tf.keras.models.Sequential()

In [240]:
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation = "relu", input_dim = number_input_features))

In [241]:
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation = "relu"))

In [242]:
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation = "relu"))

In [243]:
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation = "relu"))

In [244]:
nn_model.add(tf.keras.layers.Dense(units=1, activation="linear"))

In [245]:
nn_model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_62 (Dense)             (None, 300)               50400     
_________________________________________________________________
dense_63 (Dense)             (None, 128)               38528     
_________________________________________________________________
dense_64 (Dense)             (None, 48)                6192      
_________________________________________________________________
dense_65 (Dense)             (None, 24)                1176      
_________________________________________________________________
dense_66 (Dense)             (None, 1)                 25        
Total params: 96,321
Trainable params: 96,321
Non-trainable params: 0
_________________________________________________________________


In [246]:
nn_model.compile(loss="mean_squared_error", optimizer="adam")
nn_model.fit(X_train, np.asarray(y_train), epochs=50)

Epoch 1/50
1668/1668 [==============================] - 4s 2ms/step - loss: 4761727270912.0000
Epoch 2/50
1668/1668 [==============================] - 4s 2ms/step - loss: 4372834287616.0000
Epoch 3/50
1668/1668 [==============================] - 4s 2ms/step - loss: 4306882265088.0000
Epoch 4/50
1668/1668 [==============================] - 4s 2ms/step - loss: 4258384576512.0000
Epoch 5/50
1668/1668 [==============================] - 4s 2ms/step - loss: 4214243983360.0000
Epoch 6/50
1668/1668 [==============================] - 4s 2ms/step - loss: 4178294603776.0000
Epoch 7/50
1668/1668 [==============================] - 4s 2ms/step - loss: 4145253711872.0000
Epoch 8/50
1668/1668 [==============================] - 4s 2ms/step - loss: 4119628349440.0000
Epoch 9/50
1668/1668 [==============================] - 4s 2ms/step - loss: 4098832728064.0000
Epoch 10/50
1668/1668 [==============================] - 4s 2ms/step - loss: 4077357891584.0000
Epoch 11/50
1668/1668 [==========================

In [247]:
y_train_pred = nn_model.predict(X_train)
y_test_pred = nn_model.predict(X_test)

In [248]:
r2_score(y_train, y_train_pred)

0.32421135947667745

In [249]:
r2_score(y_test, y_test_pred)

0.2946384223801013